In [23]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json

In [26]:
df_races = pd.read_json("races_new_test.json")
links = df_races['Link'].unique()

In [32]:
allRows = []
for link in links[:5]:
    url = f"https://scores.collegesailing.org/sailors/{link}/"
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')

    sailorData = json.loads(soup.find('meta',attrs={'name': 'ts:data'})['content'])
    allRows.append([link, sailorData['name'],sailorData['first_name'],sailorData['last_name'],sailorData['gender'],sailorData['year'],sailorData['school'].split("/")[2], sailorData['id'], sailorData['external_id']])
    
df_sailor_info = pd.DataFrame(allRows, columns=['link', 'name', 'first_name', 'last_name', 'gender', 'year', 'school','id', 'external_id'])
df_sailor_info

,link,name,first_name,last_name,gender,year,school,id,external_id
0,campbell-deliscu,Campbell D'Eliscu,Campbell,D'Eliscu,M,2019,georgetown,3112221,27830
1,sean-segerblom,Sean Segerblom,Sean,Segerblom,M,2020,georgetown,3117220,29802
2,meaghan-macrae,Meaghan MacRae,Meaghan,MacRae,F,2018,georgetown,3109481,25661
3,claire-mohun,Claire Mohun,Claire,Mohun,F,2019,georgetown,3115481,28785
4,steven-leuck,Steven Leuck,Steven,Leuck,M,2020,charleston,3117488,30344


In [34]:
sailor_info = pd.read_json("sailor_data.json")
sailor_info

,link,name,first_name,last_name,gender,year,school,id,external_id
0,campbell-deliscu,Campbell D'Eliscu,Campbell,D'Eliscu,M,2019,georgetown,3112221,27830
1,sean-segerblom,Sean Segerblom,Sean,Segerblom,M,2020,georgetown,3117220,29802
2,meaghan-macrae,Meaghan MacRae,Meaghan,MacRae,F,2018,georgetown,3109481,25661
3,claire-mohun,Claire Mohun,Claire,Mohun,F,2019,georgetown,3115481,28785
4,steven-leuck,Steven Leuck,Steven,Leuck,M,2020,charleston,3117488,30344
...,...,...,...,...,...,...,...,...,...
95,gabrielle-rigby,Gabrielle Rigby,Gabrielle,Rigby,F,2019,western-washington,3115284,28641
96,cynthia-sovulewski,Cynthia Sovulewski,Cynthia,Sovulewski,F,2019,western-washington,3111104,27012
97,piper-wolters,Piper Wolters,Piper,Wolters,F,2020,western-washington,3118299,30920
98,vincent-todosiev,Vincent Todosiev,Vincent,Todosiev,M,2019,berkeley,3116331,29364
